In [4]:
#############################################################################################
# Librairies
#############################################################################################

import pandas as pd
import numpy as np
#import pylab as pl
import sys
sys.path.append("C:\\Users\\dgr\\OneDrive - Publicis Groupe\\Documents\\Formations\\SCU Python\\NEW\\modules")
sys.path
#import module_lift as mdl
import os
repertoire = "C:\\Users\\dgr\\OneDrive - Publicis Groupe\\Documents\\Formations\\SCU Python\\NEW\\data"
os.chdir(repertoire)

## Data : score attrition Telco

In [5]:
_churn_df = pd.read_csv('churn.csv')
_churn_df.dtypes
# nb de lignes 3333
_churn_df.shape[0]
# nb de vars 21
_churn_df.shape[1]
_churn_df.info()
_churn_df.columns
churn_df=_churn_df
churn_df['Area Code']=churn_df['Area Code'].astype(str)
stats = churn_df.describe(include='all')
# On récupère les colonnes et on les stocke dans une liste
col_names = churn_df.columns.tolist()
print( "Column names:")
print(col_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   object 
 5   VMail Plan      3333 non-null   object 
 6   VMail Message   3333 non-null   int64  
 7   Day Mins        3333 non-null   float64
 8   Day Calls       3333 non-null   int64  
 9   Day Charge      3333 non-null   float64
 10  Eve Mins        3333 non-null   float64
 11  Eve Calls       3333 non-null   int64  
 12  Eve Charge      3333 non-null   float64
 13  Night Mins      3333 non-null   float64
 14  Night Calls     3333 non-null   int64  
 15  Night Charge    3333 non-null   float64
 16  Intl Mins       3333 non-null   float64
 17  Intl Calls      3333 non-null   i

In [6]:
# Création d'un vecteur y contenant la variable cible binarisée
y = np.array((churn_df['Churn?'] == 'True.'))
# ou
#y = np.where(churn_df['Churn?'] == 'True.',1,0)
print(y.mean())
# 14.5% de churners


0.14491449144914492


In [7]:
pd.crosstab(y, columns = 'count')


col_0,count
row_0,
False,2850
True,483


In [8]:
#	En %
print(pd.crosstab(y, columns = 'count', normalize=True))
# col area
print(pd.crosstab(churn_df['Area Code'],y, normalize='index'))
#print(pd.crosstab(churn_df['Area Code'],y).apply(lambda r: r/r.sum(), axis=1))
# flat mais pour la pédagogie on va quand même l'encoder

col_0     count
row_0          
False  0.855086
True   0.144914
col_0         False     True 
Area Code                    
408        0.854415  0.145585
415        0.857402  0.142598
510        0.851190  0.148810


In [9]:

# par contre les deux var qui indiquent si le client paie pour un service international ou VM sont très discriminantes
print(pd.crosstab(churn_df["Int'l Plan"],y, normalize='index'))
print(pd.crosstab(churn_df["VMail Plan"],y, normalize='index'))

col_0          False     True 
Int'l Plan                    
no          0.885050  0.114950
yes         0.575851  0.424149
col_0          False     True 
VMail Plan                    
no          0.832849  0.167151
yes         0.913232  0.086768


In [10]:
 #  les dummies
pd.get_dummies(churn_df['Area Code'],prefix='area_', drop_first=True)
# jointure selon l'index des lignes (par défaut )
churn_df = pd.concat([churn_df,pd.get_dummies(churn_df['Area Code'],prefix='area_', drop_first=True)],axis=1)
#  drop l'original
churn_df.drop(['Area Code'],axis=1, inplace=True)

In [11]:
del _churn_df , repertoire
#from sklearn.preprocessing import OneHotEncoder
#onehot_encoder = OneHotEncoder(sparse=False)
#area_2d = np.array(churn_df['Area Code']).reshape(len(churn_df['Area Code']), 1)
#area = onehot_encoder.fit_transform(area_2d)

In [12]:


# étude du state
#print(pd.crosstab(churn_df['State'],y))
print(pd.crosstab(churn_df['State'],y,normalize='index'))

#print(pd.crosstab(churn_df['State'],y).apply(lambda r: r/r.sum(), axis=1))


col_0     False     True 
State                    
AK     0.942308  0.057692
AL     0.900000  0.100000
AR     0.800000  0.200000
AZ     0.937500  0.062500
CA     0.735294  0.264706
CO     0.863636  0.136364
CT     0.837838  0.162162
DC     0.907407  0.092593
DE     0.852459  0.147541
FL     0.873016  0.126984
GA     0.851852  0.148148
HI     0.943396  0.056604
IA     0.931818  0.068182
ID     0.876712  0.123288
IL     0.913793  0.086207
IN     0.873239  0.126761
KS     0.814286  0.185714
KY     0.864407  0.135593
LA     0.921569  0.078431
MA     0.830769  0.169231
MD     0.757143  0.242857
ME     0.790323  0.209677
MI     0.780822  0.219178
MN     0.821429  0.178571
MO     0.888889  0.111111
MS     0.784615  0.215385
MT     0.794118  0.205882
NC     0.838235  0.161765
ND     0.903226  0.096774
NE     0.918033  0.081967
NH     0.839286  0.160714
NJ     0.735294  0.264706
NM     0.903226  0.096774
NV     0.787879  0.212121
NY     0.819277  0.180723
OH     0.871795  0.128205
OK     0.852

In [13]:
len(churn_df['State'].unique())
# 51 valeurs
# soit on garde et on crée des dummy
# soit on fait une stratégie qui consiste à remplacer la modalité par le taux de 1
# par contre il faut le faire sur l'ech dapp sinon on triche


51

## Feature engineering sur la variable STATE

In [15]:

# split Apprentissage Test
from sklearn.model_selection import train_test_split 
state_train, state_test, y_train, y_test = train_test_split(churn_df['State'],y, test_size=0.4)

In [16]:
taux_reponse_state_train=pd.crosstab(state_train,y_train).apply(lambda r: r/r.sum(), axis=1)[1]
taux_reponse_state_test=pd.crosstab(state_test,y_test).apply(lambda r: r/r.sum(), axis=1)[1]
del state_test,y_test,  taux_reponse_state_test
del state_train

In [17]:
# on remplace le state par le taux de reponse associé
list(taux_reponse_state_train)
# replace une liste par une liste
churn_df['State'].replace(list(taux_reponse_state_train.index), list(taux_reponse_state_train), inplace=True)
churn_df.rename(columns={"State": "churnMoy_state"},inplace=True)

In [18]:


# X
to_drop=['Phone','Churn?']
churn_X = churn_df.drop(to_drop,axis=1)
churn_X.describe()
# 'yes'/'no' has to be converted to boolean values
# NumPy converts these from boolean to 1. and 0. later
yes_no_cols = ["Int'l Plan","VMail Plan"]
churn_X[yes_no_cols] = churn_X[yes_no_cols] == 'yes'


In [19]:
# supprime les objets intermédiaires
del to_drop,yes_no_cols

In [20]:



# Stocke les noms des X 
features = list(churn_X.columns)
churn_X.dtypes

churnMoy_state    float64
Account Length      int64
Int'l Plan           bool
VMail Plan           bool
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls          int64
Intl Charge       float64
CustServ Calls      int64
area__415           uint8
area__510           uint8
dtype: object

In [21]:
# Normalisation 
from sklearn.preprocessing import StandardScaler

# centrer réduire (données doivent être float et pas int)
norm=StandardScaler().fit(churn_X.astype(float))

# on applique le centrage réduction
X = norm.transform(churn_X.astype(float))

# on aurait pu écrire : X=StandardScaler().fit_transform(churn_X.astype(float))

In [22]:
# X en dataframe
X = pd.DataFrame(X)
# on récupère les noms de col
X.columns = churn_X.columns
X.dtypes
print ("Feature space holds %d observations and %d features" % X.shape)
print ( "Unique target labels:", np.unique(y))

del taux_reponse_state_train,churn_X,y_train

Feature space holds 3333 observations and 20 features
Unique target labels: [False  True]


## Partition Apprentissage Test

In [24]:

# split Apprentissage Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4,random_state=123)


## Analyse distribution des variables

In [26]:

# 14.4% de churners
y.mean()
X.describe()

,churnMoy_state,Account Length,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,area__415,area__510
count,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03,3.333000e+03
mean,3.694415e-16,1.713134e-16,-1.021951e-16,6.921823e-17,3.832651e-16,7.081711e-16,-2.150828e-16,-3.226409e-16,-1.067919e-16,3.527531e-16,1.266447e-16,6.602046e-17,-7.041739e-17,-3.134473e-17,-3.274375e-16,-1.264615e-16,2.785384e-16,3.275708e-16,8.076348e-16,3.511209e-16
std,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00,1.000150e+00
min,-1.882355e+00,-2.513172e+00,-3.275805e-01,-6.183963e-01,-5.917599e-01,-3.301096e+00,-5.005247e+00,-3.301162e+00,-3.963622e+00,-5.025911e+00,-3.963679e+00,-3.513648e+00,-3.429870e+00,-3.515366e+00,-3.667413e+00,-1.820289e+00,-3.668210e+00,-1.188218e+00,-9.931230e-01,-5.804683e-01
25%,-6.730589e-01,-6.797448e-01,-3.275805e-01,-6.183963e-01,-5.917599e-01,-6.624241e-01,-6.695701e-01,-6.623760e-01,-6.780300e-01,-6.583610e-01,-6.783123e-01,-6.698545e-01,-6.699340e-01,-6.676792e-01,-6.223690e-01,-6.011951e-01,-6.164341e-01,-4.279320e-01,-9.931230e-01,-5.804683e-01
50%,-2.627621e-01,-1.627644e-03,-3.275805e-01,-6.183963e-01,-5.917599e-01,-6.887677e-03,2.812491e-02,-6.730063e-03,8.276141e-03,-5.738630e-03,8.459274e-03,6.485803e-03,-5.505089e-03,4.691242e-03,2.246393e-02,-1.948306e-01,2.045823e-02,-4.279320e-01,-9.931230e-01,-5.804683e-01
75%,6.237517e-01,6.513740e-01,-3.275805e-01,1.617086e+00,8.695542e-01,6.725198e-01,6.759846e-01,6.726790e-01,6.768330e-01,6.970854e-01,6.766695e-01,6.808485e-01,6.589239e-01,6.814562e-01,6.672969e-01,6.178983e-01,6.706192e-01,3.323545e-01,1.006925e+00,1.722747e+00
max,3.064764e+00,3.564766e+00,3.052685e+00,1.617086e+00,3.134591e+00,3.140422e+00,3.217588e+00,3.140803e+00,3.209066e+00,3.508382e+00,3.207980e+00,3.839081e+00,3.827739e+00,3.836763e+00,3.497397e+00,6.307001e+00,3.496829e+00,5.654360e+00,1.006925e+00,1.722747e+00


In [27]:


#concatenation X et y pour utiliser la fonction boxplot
Xy = pd.concat([X,pd.DataFrame(y)], axis = 1)
# noms des variables 
Xy.columns=features+['Churn?']


In [28]:
import pickle
file=open("churn_prepared.pydata","wb")
pickle.dump(Xy,file)
file.close()

In [ ]:
#box plot
#Xy.boxplot(column = ['Account Length'], by = 'Churn?')
Xy.boxplot(column = features[0:6], by = 'Churn?')
Xy.boxplot(column = features[6:12], by = 'Churn?')
Xy.boxplot(column = features[12:], by = 'Churn?')

In [30]:
# tri croisé des var quali

for i in ("Int'l Plan","VMail Plan","area__415","area__510"):
     print(pd.crosstab(X[""+str(i)],y))
     print(pd.crosstab(X[""+str(i)],y).apply(lambda r: r/r.sum(), axis=1))

col_0       False  True 
Int'l Plan              
-0.327580    2664    346
 3.052685     186    137
col_0          False     True 
Int'l Plan                    
-0.327580   0.885050  0.114950
 3.052685   0.575851  0.424149
col_0       False  True 
VMail Plan              
-0.618396    2008    403
 1.617086     842     80
col_0          False     True 
VMail Plan                    
-0.618396   0.832849  0.167151
 1.617086   0.913232  0.086768
col_0      False  True 
area__415              
-0.993123   1431    247
 1.006925   1419    236
col_0         False     True 
area__415                    
-0.993123  0.852801  0.147199
 1.006925  0.857402  0.142598
col_0      False  True 
area__510              
-0.580468   2135    358
 1.722747    715    125
col_0         False     True 
area__510                    
-0.580468  0.856398  0.143602
 1.722747  0.851190  0.148810
